In [51]:
!pip install pandas

In [52]:
!pip install sklearn

In [53]:
import os
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, r2_score
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing

In [54]:
# lakukan load dataset terlebih dahulu
import pandas as pd
car = pd.read_csv("drive/My Drive/used_car_data.csv")
car.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti Wagon R LXI CNG,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


PREPROCESS DATA

In [55]:
data_clean = car.copy(deep = True)

Mari kita periksa bagaimana dataframe akan terlihat tanpa nilai NaN.

In [56]:
data_clean.dropna("index").shape

(5975, 12)

Karena nilai Nama yang unik menghasilkan hampir sepertiga dari semua titik data, mari letakkan model mobil dan simpan hanya mobil yang dibuat (Audi, BMW, dll.). Ini akan membantu untuk menghindari masalah overfitting, artinya model kami tidak akan terlalu bergantung pada Nama mobil untuk membuat prediksi.

In [57]:
names = list(data_clean.Name)
for i in range(len(names)):
    names[i] = names[i].split(' ', 1)[0]
data_clean.Name = names
data_clean.head()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,Price
0,Maruti,Mumbai,2010,72000,CNG,Manual,First,26.6 km/kg,998 CC,58.16 bhp,5.0,1.75
1,Hyundai,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5.0,12.50
2,Honda,Chennai,2011,46000,Petrol,Manual,First,18.2 kmpl,1199 CC,88.7 bhp,5.0,4.50
3,Maruti,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7.0,6.00
4,Audi,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5.0,17.74


Seperti yang Anda lihat, sekarang kami hanya memiliki 31 mobil unik.

In [58]:
len(np.unique(list(data_clean.Name)))

31

In [59]:
data_clean['Name'].value_counts()

Maruti           1211
Hyundai          1107
Honda             608
Toyota            411
Mercedes-Benz     318
Volkswagen        315
Ford              300
Mahindra          272
BMW               267
Audi              236
Tata              186
Skoda             173
Renault           145
Chevrolet         121
Nissan             91
Land               60
Jaguar             40
Fiat               28
Mitsubishi         27
Mini               26
Volvo              21
Porsche            18
Jeep               15
Datsun             13
Force               3
ISUZU               2
Ambassador          1
Lamborghini         1
Bentley             1
Isuzu               1
Smart               1
Name: Name, dtype: int64

In [60]:
data_clean['Location'].value_counts()

Mumbai        790
Hyderabad     742
Kochi         651
Coimbatore    636
Pune          622
Delhi         554
Kolkata       535
Chennai       494
Jaipur        413
Bangalore     358
Ahmedabad     224
Name: Location, dtype: int64

In [61]:
data_clean['Year'].value_counts()

2014    797
2015    744
2016    741
2013    649
2017    587
2012    580
2011    466
2010    342
2018    298
2009    198
2008    174
2007    125
2019    102
2006     78
2005     57
2004     31
2003     17
2002     15
2001      8
1998      4
2000      4
1999      2
Name: Year, dtype: int64

In [62]:
# merubah mileage ke str
car['Mileage'].astype(str)

0       26.6 km/kg
1       19.67 kmpl
2        18.2 kmpl
3       20.77 kmpl
4        15.2 kmpl
           ...    
6014     28.4 kmpl
6015     24.4 kmpl
6016     14.0 kmpl
6017     18.9 kmpl
6018    25.44 kmpl
Name: Mileage, Length: 6019, dtype: object

In [63]:
# menghapus 4 kata dari belakang mileage
car['Mileage'] = pd.to_numeric(car['Mileage'].astype(str).str[:-4], errors='coerce').replace(np.nan, 0)

In [64]:
# merubah engine ke str
car['Engine'].astype(str)

0        998 CC
1       1582 CC
2       1199 CC
3       1248 CC
4       1968 CC
         ...   
6014    1248 CC
6015    1120 CC
6016    2498 CC
6017     998 CC
6018     936 CC
Name: Engine, Length: 6019, dtype: object

In [65]:
# menghapus 2 kata dari belakang engine
car['Engine'] = pd.to_numeric(car['Engine'].astype(str).str[:-2], errors='coerce').replace(np.nan, 0)

In [66]:
# merubah power ke str
car['Power'].astype(str)

0       58.16 bhp
1       126.2 bhp
2        88.7 bhp
3       88.76 bhp
4       140.8 bhp
          ...    
6014       74 bhp
6015       71 bhp
6016      112 bhp
6017     67.1 bhp
6018     57.6 bhp
Name: Power, Length: 6019, dtype: object

In [68]:
# menghapus 3 kata dari belakang power
car['Power'] = pd.to_numeric(car['Power'].astype(str).str[:-3], errors='coerce').replace(np.nan, 0)

In [69]:
# mengubah rate
label = []
for index, row in car.iterrows():
    if row["Transmission"] == "Automatic":
        label.append(1)
    else:
        label.append(0)

car["Transmission1"] = label
car = car.drop(columns=['Transmission'])
car.tail()

,Name,Location,Year,Kilometers_Driven,Fuel_Type,Owner_Type,Mileage,Engine,Power,Seats,Price,Transmission1
6014,Maruti Swift VDI,Delhi,2014,27365,Diesel,First,28.40,1248.0,7.0,5.0,4.75,0
6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,Diesel,First,24.40,1120.0,7.0,5.0,4.00,0
6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Diesel,Second,14.00,2498.0,11.0,8.0,2.90,0
6017,Maruti Wagon R VXI,Kolkata,2013,46000,Petrol,First,18.90,998.0,6.0,5.0,2.65,0
6018,Chevrolet Beat Diesel,Hyderabad,2011,47000,Diesel,First,25.44,936.0,5.0,5.0,2.50,0


In [70]:
# mengubah rate
label = []
for index, row in car.iterrows():
    if row["Fuel_Type"] == "Petrol":
        label.append(0)
    elif row["Fuel_Type"] == "Diesel":
        label.append(1)
    elif row["Fuel_Type"] == "Electric":
        label.append(2)
    elif row["Fuel_Type"] == "CNG":
        label.append(3)
    else:
        label.append(4)

car["Fuel_Type1"] = label
car = car.drop(columns=['Fuel_Type'])
car.tail()

,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,Price,Transmission1,Fuel_Type1
6014,Maruti Swift VDI,Delhi,2014,27365,First,28.40,1248.0,7.0,5.0,4.75,0,1
6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,First,24.40,1120.0,7.0,5.0,4.00,0,1
6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,Second,14.00,2498.0,11.0,8.0,2.90,0,1
6017,Maruti Wagon R VXI,Kolkata,2013,46000,First,18.90,998.0,6.0,5.0,2.65,0,0
6018,Chevrolet Beat Diesel,Hyderabad,2011,47000,First,25.44,936.0,5.0,5.0,2.50,0,1


In [71]:
# mengubah rate
label = []
for index, row in car.iterrows():
    if row["Owner_Type"] == "First":
        label.append(1)
    elif row["Owner_Type"] == "Second":
        label.append(2)
    elif row["Owner_Type"] == "Thrid":
        label.append(3)
    else:
        label.append(4)

car["Owner_Type1"] = label
car = car.drop(columns=['Owner_Type'])
car.tail()

,Name,Location,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Transmission1,Fuel_Type1,Owner_Type1
6014,Maruti Swift VDI,Delhi,2014,27365,28.40,1248.0,7.0,5.0,4.75,0,1,1
6015,Hyundai Xcent 1.1 CRDi S,Jaipur,2015,100000,24.40,1120.0,7.0,5.0,4.00,0,1,1
6016,Mahindra Xylo D4 BSIV,Jaipur,2012,55000,14.00,2498.0,11.0,8.0,2.90,0,1,2
6017,Maruti Wagon R VXI,Kolkata,2013,46000,18.90,998.0,6.0,5.0,2.65,0,0,1
6018,Chevrolet Beat Diesel,Hyderabad,2011,47000,25.44,936.0,5.0,5.0,2.50,0,1,1


In [72]:
names = list(data_clean.Name)
for i in range(len(names)):
    names[i] = names[i].split(' ', 1)[0]
car.Name = names
car.head()

,Name,Location,Year,Kilometers_Driven,Mileage,Engine,Power,Seats,Price,Transmission1,Fuel_Type1,Owner_Type1
0,Maruti,Mumbai,2010,72000,0.00,998.0,58.0,5.0,1.75,0,3,1
1,Hyundai,Pune,2015,41000,19.67,1582.0,12.0,5.0,12.50,0,1,1
2,Honda,Chennai,2011,46000,18.20,1199.0,8.0,5.0,4.50,0,0,1
3,Maruti,Chennai,2012,87000,20.77,1248.0,88.0,7.0,6.00,0,1,1
4,Audi,Coimbatore,2013,40670,15.20,1968.0,14.0,5.0,17.74,1,1,2


In [73]:
# Dropping unnecessary columns
car.drop(['Name', 'Location', 'Year',  ],axis=1,inplace=True)

In [74]:
# Dropping missing values rows
car.dropna(inplace=True)

In [75]:
y = car[["Price"]].to_numpy()
car = car.drop(columns = ["Price"])

In [76]:
x = car.values
columns = car.columns
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
car = pd.DataFrame(x_scaled)
car.columns = columns

In [78]:
X = car.to_numpy()

In [79]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.9, test_size=0.1, random_state=1)

Now, let's fit the model and check accuracy on the training data.

In [80]:
lm = LinearRegression(fit_intercept=True, normalize=True, copy_X=True)
lm.fit(X_train, y_train)
print('Training accuracy (the coefficient of determination): {:.3f}'.format(lm.score(X_train, y_train)))

Training accuracy (the coefficient of determination): 0.571


Now we could test the model on the test data.

In [82]:
prediction = lm.predict(X_test)

In [83]:
print('Testing accuracy by using score function: {:.3f}'.format(lm.score(X_test, y_test)))
print('Testing accuracy by using r2_score meric: {:.3f}'.format(r2_score(y_test, lm.predict(X_test))))

Testing accuracy by using score function: 0.563
Testing accuracy by using r2_score meric: 0.563


Seperti yang Anda lihat, akurasi pengujian sangat mirip dengan akurasi pelatihan, jadi kita dapat mengatakan bahwa model tersebut tidak sesuai dengan data pelatihan. Mari kita gunakan metrik yang lebih intuitif - kesalahan absolut rata-rata (mean absolute error) - untuk mengevaluasi kinerja model.

In [84]:
mean_absolute_error(y_test, prediction)

4.503027850162978

Model kami berkinerja cukup baik. Rata-rata, itu akan salah menentukan harga mobil sekitar 4,50 INR Lakh. 